In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
feature_list = ['x','y','hour','weekday','day','month','year', 'accuracy']

In [3]:
def prepare_data(df, n_cell_x, n_cell_y):
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_x'] = pos_x
    df['grid_y'] = pos_y
    
    #Feature engineering
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def run_prediction(df_train, df_test, n_cell_x, n_cell_y, th):
    total_result = pd.DataFrame()
    for x_grid in range(0, n_cell_x):
        start_time = time.time()
        for y_grid in range(0, n_cell_y):
            total_result = total_result.append(process_one_cell(df_train, df_test, x_grid, y_grid, th))
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_grid, flush = True)
    return total_result

In [5]:
def process_one_cell(df_train, df_test, grid_x, grid_y, th):
    start_time = time.time()
    
    df_cell_train = df_train[((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y))]# |
    '''
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y - 1))]
    '''
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    df_cell_test = df_test[(df_test.grid_x == grid_x)&(df_test.grid_y == grid_y)]
    row_ids = df_cell_test.row_id
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=35, weights='distance', 
                               metric='euclidean')
    clf.fit(df_cell_train[feature_list], df_cell_train.place_id)
    predictions = clf.predict_proba(df_cell_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_cell_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [6]:
def run_validation():
    # Run validation
    df_train = pd.read_csv('../../train.csv',
                               usecols=['row_id','x','y','accuracy','time','place_id'])
    
    df_valiation = df_train[df_train.time > 786239 * 0.875]
    
    df_train = df_train[df_train.time <= 786239 * 0.875]
    
    n_cell_x = 10
    n_cell_y = 20 
    
    print('Preparing train data')
    df_train = prepare_data(df_train, n_cell_x, n_cell_y)
    df_valiation = prepare_data(df_valiation, n_cell_x, n_cell_y)
    df_valiation['p1'] = np.nan
    df_valiation['p2'] = np.nan
    df_valiation['p3'] = np.nan
    
    prediction_result = run_prediction(df_train, df_valiation, n_cell_x, n_cell_y, 5)
    
    # Calculate score
    prediction_result.sort_index(inplace=True)
    prediction_result['score'] = (prediction_result.p1 == df_valiation.place_id) * 1
    prediction_result['score'] += (prediction_result.p2 == df_valiation.place_id) * 0.5
    prediction_result['score'] += (prediction_result.p3 == df_valiation.place_id) * 0.33
    
    print(prediction_result.score.describe(), flush=True)

In [7]:
print('x==============', flush=True)
fw = [360, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [380, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [420, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [440, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
print('y==============', flush=True)
fw = [400, 900, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 950, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1050, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1100, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
print('hour==============', flush=True)
fw = [400, 1000, 3.6, 3.15, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 3.8, 3.3, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4.2, 3.7, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4.4, 3.85, 1./22., 2, 9, 0.09]
run_validation()
print('weekday==============', flush=True)
fw = [400, 1000, 4, 3.5, 1./18., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./20., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./24., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./26., 2, 9, 0.09]
run_validation()
print('month==============', flush=True)
fw = [400, 1000, 4, 3.5, 1./22., 1.8, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 1.9, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2.1, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2.2, 9, 0.09]
run_validation()
print('year==============', flush=True)
fw = [400, 1000, 4, 3.5, 1./22., 2, 8.1, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 8.55, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9.45, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9.9, 0.09]
run_validation()
print('accuracy==============', flush=True)
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.07]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.08]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.1]
run_validation()
fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.11]
run_validation()

x==============
Preparing train data
Elapsed time overall: 51.03383660316467 seconds 0
Elapsed time overall: 55.673367500305176 seconds 1
Elapsed time overall: 59.51832914352417 seconds 2
Elapsed time overall: 61.851274490356445 seconds 3
Elapsed time overall: 61.751662731170654 seconds 4
Elapsed time overall: 63.39539074897766 seconds 5
Elapsed time overall: 61.12460780143738 seconds 6
Elapsed time overall: 63.63538122177124 seconds 7
Elapsed time overall: 61.566190242767334 seconds 8
Elapsed time overall: 56.37935733795166 seconds 9
count    4417684.000000
mean           0.528209
std            0.451802
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64
Preparing train data
Elapsed time overall: 55.93660092353821 seconds 0
Elapsed time overall: 58.74470615386963 seconds 1
Elapsed time overall: 62.9845232963562 seconds 2
Elapsed time overall: 58.03403973579407 seconds 3
Elapsed time overal

Validation result: 0.539376

In [ ]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_validation_result.csv', index=False)

In [ ]:
# Run test
df_train = pd.read_csv('../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 30
n_cell_y = 60 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_result.csv', index=False)